# 潛在語意，字詞背後隱藏的概念


* 有些字詞會共同出現，為什麼字詞會經常共同出現，很可能有特殊的意義，此即所謂的潛在語意。
* 本學習包先將本本降維，再依每個字詞在不同的維度上有投影量，形成該字詞的詞向量，計算不同詞向量的距離，找出距離較近的一些字詞，藉以判斷潛在語意。
* 亦可用詞對分析，達到同樣的效果。
* word2vec 以類神經網絡求得各字詞的向量。有結構的相似性的字詞，向量較接近。例如馬英九、陳水扁、蔡英文會很接近，雖然未必共現，亦非同義語，但他們經常與「總統」共同出現，有結構的相似性。

# lsa 潛在語意分析

In [27]:
from string import *
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import numpy as np
from scipy import spatial
import pandas as pd

In [25]:
# 讀取檔案 #######################################
import pandas as pd
# 必須要有的參數
unit="arti"
field="tfidf"
path="D:/my python/people/" 
title="all"
fname=path+"working/"+unit+"_"+field+"_"+title+".pkl"
df=pd.read_pickle(fname)

##  extract_feature(num_i, doc_list,df)
* 僅保留 tfidf 值較大者，簡化文本


In [26]:
# doc_list=df.token
def extract_feature(num_i, doc_list,df):
    # tfidf 加權
    transformer = TfidfVectorizer()
    tfidf = transformer.fit_transform(doc_list)     # sparse matrix
    # 降維
    svd = TruncatedSVD(num_i, algorithm = 'arpack')   # object
    dtm_lsa = svd.fit_transform(tfidf)  
    dtm_com = svd.fit(tfidf)  # object, not numpy array
    # components
    comp=dtm_com.components_
    print("維度數",len(comp))
    print("字詞數",len(comp[0]))

    # 如果文章筆數、字詞數太多，會超過記憶體 error
    dtm_tfidf=tfidf.toarray() 
    # 字詞的編號
    terms=vectorizer.vocabulary_
    # 字詞的 list
    term_list=vectorizer.get_feature_names_out()
    # 轉成 DataFrame
    df_tfidf=pd.DataFrame(dtm_tfidf, columns=term_list)

    # 只保留 TFIDF 太於 0.05 者，簡化字詞數，否則矩陣太大
    m = 0.05
    featureset = []
    for i in range(len(df_tfidf)):    
        # 所有字詞，在該篇文章中，tfidf 大於 m 為 true，否則為 false
        cri = df_tfidf.iloc[i]>=m
        # 合乎條件的字詞 list
        te = [x for j,x in enumerate(term_list) if cri[j]]    
        # 合乎條件的 tfidf list
        tf= [x for j,x in enumerate(df_tfidf.iloc[i]) if cri[j]]        
        # 合於條件的字詞改為空白隔開的string
        features_string=" ".join(te)     
        # 一篇文章一個字詞的 list
        featureset.append(features_string) 
    df['feature']=fetureset   
    return df


## 最接近的字詞

In [28]:
def concept(path, df, token, num_i):
    # tfidf 加權
    num_i=50
    doc_list=df[token].str.replace(","," ")

    # Term document frequency 轉換，成為 matrix ,
    transformer = TfidfVectorizer()
    tfidf = transformer.fit_transform(doc_list)     # sparse matrix
    # 字詞的編號
    terms=transformer.vocabulary_
    # 字詞的 list
    term_list=transformer.get_feature_names_out()
    
    # latent semantic analysis, 變數縮減
    svd = TruncatedSVD(num_i, algorithm = 'arpack')   # object
    lsa = svd.fit_transform(tfidf.T)  # numpy array, (number of terms * num_i)     

    dtm_lsa = svd.fit_transform(tfidf)  
    dtm_com = svd.fit(tfidf)  # object, not numpy array
    # components
    comp=dtm_com.components_
    print("維度數",len(comp))
    print("字詞數",len(comp[0]))
    
    # 每個維度軸都是各字詞的線性組合，每個維度的構成
    df_concept=pd.DataFrame(comp,columns= term_list)
    df_concept=df_concept.transpose()
    # 各維度字詞，依重要性大小排序
    con=[]
    sig=[]
    for n in range(len(df_concept.columns)):
        # 由大至小排序
        df_order= df_concept.sort_values(by=n, ascending=False)
        con.append(list(df_order.index)[0:500])
        sig.append(list(df_order[n])[0:500])
    # 印出各軸的重要字詞
    for n in range(len(con)):
        print (n,con[n][:30])          
    # 轉成 dataframe 格式
    df_concept=pd.DataFrame(con).T
    return df_concept, transformer, lsa, term_list

#  最接近的 k 個
def similarity(a,b):
    result = 1 - spatial.distance.cosine(a, b)
    return result
def kClosestTerms(k,term,transformer,lsa,term_list):     
    # 字詞的編號
    index = transformer.vocabulary_[term] 
    # 該字詞與其他字詞的相似度
    similar=[]
    for i in range(len(lsa)):
        s=similarity(lsa[index],lsa[i])
        similar.append(s) 
        if i%10000==0:
            print (i,"done",end=" ")
    closestTerms = {}   # dictionary
    for i in range(len(lsa)):
        closestTerms[term_list[i]] = similar[i]    
        if i%10000==0:
            print (i,"done",end=" ")                   
    sortedList = sorted(closestTerms , key= lambda l : closestTerms[l])
    return sortedList[::-1][0:k]

In [10]:
df_concept=concept(tfidf, transformer,term_list)
df_concept.iloc[:20]

維度數 10
字詞數 1967779
0 ['中国', '发展', '国家', '经济', '北京', '建设', '国际', '合作', '政府', '关系', '会议', '美国', '全国', '访问', '代表团', '组织', '支持', '世界', '社会', '领导', '和平', '地区', '政治', '生产', '会见', '解决', '斗争', '政策', '提高', '总统']
1 ['生产', '提高', '建设', '群众', '干部', '企业', '领导', '部门', '管理', '农村', '全国', '思想', '学习', '组织', '农民', '教育', '服务', '农业', '技术', '改革', '社会', '制度', '职工', '增长', '产品', '经验', '市场', '质量', '生活', '研究']
2 ['斗争', '反对', '革命', '侵略', '胜利', '美帝国主义', '帝国主义', '美国', '战争', '毛主席', '团结', '谴责', '无产阶级', '和平', '阴谋', '全世界', '民族', '战斗', '苏联', '越南', '独立', '声明', '敌人', '修正主义', '军队', '解放', '力量', '民主', '群众', '毛泽东思想']
3 ['革命', '同志', '毛主席', '代表团', '学习', '思想', '无产阶级', '群众', '领导', '率领', '中国', '团长', '友好', '干部', '会见', '毛泽东思想', '北京', '乘飞机', '访问', '大使', '友谊', '胜利', '今天下午', '修正主义', '教导', '文化大革命', '教育', '到达', '友好访问', '参观']
4 ['发展', '关系', '合作', '两国', '经济', '政治', '领域', '两国关系', '友好合作', '稳定', '社会', '维护', '努力', '建设', '中方', '交流', '两国人民', '会见', '共同努力', '深化', '和平', '建交', '互利', '交往', '思想', '改革开放', '江泽民', '事业', '政策', '双边关系']
5 ['比赛', '中国', '世界', 

,0,1,2,3,4,5,6,7,8,9
0,中国,生产,斗争,革命,发展,比赛,会议,美帝国主义,传真照片,传真照片
1,发展,提高,反对,同志,关系,中国,社会,企业,两国人民,经济
2,国家,建设,革命,毛主席,合作,世界,制度,建设,10,国际
3,经济,群众,侵略,代表团,两国,北京,政府,帝国主义,附图片,企业
4,北京,干部,胜利,学习,经济,选手,代表团,民族,友谊,乘飞机
5,建设,企业,美帝国主义,思想,政治,举办,中央,侵略,农民,无产阶级
6,国际,领导,帝国主义,无产阶级,领域,冠军,乘飞机,友谊,两国,美国
7,合作,部门,美国,群众,两国关系,国际,法律,越南,总理,革命
8,政府,管理,战争,领导,友好合作,女子,监督,斗争,地区,北京
9,关系,农村,毛主席,率领,稳定,运动员,北京,全世界,群众,产品


In [29]:
k=50
term='伟大领袖'
kClosestTerms(k,term,transformer,lsa,term_list)

0 done 10000 done 20000 done 30000 done 40000 done 50000 done 60000 done 70000 done 80000 done 90000 done 100000 done 110000 done 120000 done 130000 done 140000 done 150000 done 160000 done 170000 done 180000 done 190000 done 200000 done 210000 done 220000 done 230000 done 240000 done 250000 done 260000 done 270000 done 280000 done 290000 done 300000 done 310000 done 320000 done 330000 done 340000 done 350000 done 360000 done 370000 done 380000 done 390000 done 400000 done 410000 done 420000 done 430000 done 440000 done 450000 done 460000 done 470000 done 480000 done 490000 done 500000 done 510000 done 520000 done 530000 done 540000 done 550000 done 560000 done 570000 done 580000 done 590000 done 600000 done 610000 done 620000 done 630000 done 640000 done 650000 done 660000 done 670000 done 680000 done 690000 done 700000 done 710000 done 720000 done 730000 done 740000 done 750000 done 760000 done 770000 done 780000 done 790000 done 800000 done 810000 done 820000 done 830000 done 840000

['伟大领袖',
 '战无不胜',
 '对帝',
 '伟大红旗',
 '红卫兵',
 '光辉思想',
 '红彤彤',
 '红代会',
 '无限热爱',
 '林副主席',
 '毛主席',
 '千秋万代',
 '铁打江山',
 '战斗历程',
 '文化大革命',
 '四十八年',
 '风雷动',
 '最红',
 '万万岁',
 '无限信仰',
 '永世',
 '一万遍',
 '永不变色',
 '光焰',
 '最最',
 '红太阳',
 '与帝',
 '宝书',
 '海枯石烂',
 '无限崇拜',
 '伟大号召',
 '语录',
 '万寿无疆',
 '东风浩荡',
 '大长',
 '指路明灯',
 '一千遍',
 '无际',
 '毛泽东思想',
 '喜讯传来',
 '千遍万遍',
 '反修',
 '十六条',
 '举得',
 '九大',
 '大风大浪',
 '革命派',
 '大灭',
 '无产阶级',
 '伟大的统帅']

# word2vec
* word2vec 以類神經網絡求得各字詞的向量。有結構的相似性，向量較接近。例如馬英九、陳水扁、蔡英文會很接近，並非其共現性，亦非同義語，而是他們經常與「總統」共同出現。
* gensim 官方網頁: https://radimrehurek.com/gensim/tut1.html
* 安裝 gensim: 
* 命令提示字元：pip install --upgrade gensim 或 easy_install -U gensim興

## 設定系統環境與暫存檔案位置

In [20]:
# 執行時記錄時間、位置與訊息。
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import numpy
import numpy as np
import matplotlib.pyplot as plt
import os
import io
import tempfile
import re
import gensim

In [20]:
# 設定儲存暫存檔的位置 temfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))
cur_dir = os.getcwd() #印出目前工作目錄
print (cur_dir)

Folder "C:\Users\User\AppData\Local\Temp" will be used to save temporary dictionary and corpus.
C:\Users\User


In [28]:

# 自動產生一個資料夾放資料檔
if not os.path.exists(cur_dir + '/word2vec_data/'):
    os.makedirs('/word2vec_data/')

## 資料重整
* 從doc_seg1 轉成 texts, 存檔為 mydocument.txt, 新增一個目錄名為 word2vec_data
* 原資料為 list, 第一層為文件，第二層為經過斷字的文章，string。
* 改為 string, 每一行代表一篇文章。

In [8]:
year=[1946,1947,1948,1949,
      1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,
      1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,
      1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,
      1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,
      1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,
      2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,
      2010,2011,2012]
path="D:/my python/people/" 

In [29]:
# 含有 kw 的方保留
sentences=[]
years=[]
filenames=[]
titles=[]
df1=pd.DataFrame()
for n in range(len(year)):
    # 選擇斷詞好的文本
    fname=str(year[n])+"_jieba.xlsx"
    df=pd.read_excel(path+"jieba/"+fname, 0)   
    df = df.fillna("")
    df=df[['token']]
    fname=cur_dir + '/word2vec_data/'+str(year[n])+'.txt'
    with open(fname, 'a', encoding='UTF-8') as f:
        dfAsString = df.to_string(header=False, index=False)
        f.write(dfAsString)
    print (year[n],"done", end=" ")    

1946 done
1947 done
1948 done
1949 done
1950 done
1951 done
1952 done
1953 done
1954 done
1955 done
1956 done
1957 done
1958 done
1959 done
1960 done
1961 done
1962 done
1963 done
1964 done
1965 done
1966 done
1967 done
1968 done
1969 done
1970 done
1971 done
1972 done
1973 done
1974 done
1975 done
1976 done
1977 done
1978 done
1979 done
1980 done
1981 done
1982 done
1983 done
1984 done
1985 done
1986 done
1987 done
1988 done
1989 done
1990 done
1991 done
1992 done
1993 done
1994 done
1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done


## 讀取多個text檔，製作文字袋 object

In [30]:
# 每行讀取，每行代表一篇文章
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname 
    def __iter__(self):
        # 開啟每個檔案
        for fname in os.listdir(self.dirname):
            # 每行每行讀取
            for line in open(os.path.join(self.dirname, fname), encoding='utf-8'):
                yield line.split()
                
# 讀取料夾內所有檔案，轉為 list 文字袋，第一層文章，第二層字詞     
# 讀到的文字袋存為 sentences, object  
sentences = MySentences(cur_dir +"/word2vec_data")  # a memory-friendly iterator            

## 訓練 (時間比較久，要有耐心)
* 輸入大文本資料，由大量的字詞前後關係，得出每個字詞周邊可能會出現其他字詞的機率分布，同時，也可以算出降維後，每個字詞在空間中的位置(向量)，計算字詞與字詞間的距離，或相對關係。
* 基本應用：
* 運用類神經網絡(或稱深度學習 deep learning)演算法。
* Word2vec works in two models CBOW and skip-gram. Let's take CBOW model,
* 非堅督式學習，訓練結果無法得知好壞，只能從後續的應用得知。
* 教學資源：

* 參數設定
* https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec
* min_count，至少出現次數，內訂為 5
* window，移動視窗，多遠距離的字詞預估(the maximum distance between the current and predicted word)，內訂為5
* size，維度的空間(the N-dimensional space)，隱藏層的神經元數目，內訂為100
* workers，多功可加快速度，須先裝 cython 才有效，內訂為3。
* hs = if 1, hierarchical softmax will be used. If 0 (default), and negative is non-zero, negative sampling will be used.
* negative is non-zero, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). Default is 5. If set to 0, no negative samping is used.
* Memory，耗用記憶體，字詞數乘以size,window, workers。

In [33]:

# 引數 sentences 為各篇文章的文字袋(注意格式)，注意其他參數設定
seed = 666
sg = 0
window_size = 10
vector_size = 100
min_count = 1
workers = 8
epochs = 5
batch_words = 10000
model1 = gensim.models.Word2Vec(sentences,
    min_count=min_count,
    vector_size=vector_size,
    workers=workers,
    epochs=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words)

In [ ]:
words1 = list(model1.wv.vocab)
print (len(words1))

## 模型儲存

In [35]:
fname=cur_dir +"\word2vec_data\w2v_model1.txt"
model1.save(fname)

In [22]:
# 也可以存成 bin 檔
model1.save(path+"working/model.bin")

## 讀取

In [17]:
##############################################
fname=cur_dir +"\word2vec_data\w2v_model1.txt"
model1 = gensim.models.Word2Vec.load(fname)  # you can continue training with the loaded model!

NameError: name 'cur_dir' is not defined

## 某個字詞的向量


In [49]:
# 字詞的向量
a=model1.wv['自由']  # numpy vector of a word
a

array([-7.4103341e-02, -2.8792534e+00, -2.9030464e+00,  1.9297727e+00,
       -4.1995859e+00,  1.3434430e+00,  5.3185129e+00,  1.8856300e+00,
        9.9386084e-01,  3.9040930e+00, -1.8884244e+00, -1.1290009e+00,
       -3.1349480e-01, -1.8501260e+00,  1.0676432e+00,  2.9322159e+00,
       -6.3640994e-01, -5.1550965e+00, -4.3209486e+00,  9.5664543e-01,
        2.1225898e+00,  5.4785538e+00,  3.2091327e+00, -4.4638085e+00,
       -5.6119280e+00, -5.5335873e-01,  7.6833628e-03, -1.7482197e+00,
       -3.3751745e+00,  4.2273159e+00,  1.0909537e+00,  4.0444779e+00,
       -7.0587349e+00,  2.3918550e+00, -2.6788953e-01,  6.6266918e-01,
       -2.3299170e+00,  5.6378150e+00,  5.7518399e-01, -8.3680096e+00,
       -7.8566394e+00, -1.7519004e+00, -2.4536273e+00,  5.0473399e+00,
        3.1660731e+00, -2.5422773e-01,  1.8931571e+00, -2.0494962e+00,
       -3.2127509e+00,  1.7837901e+00, -8.7715740e+00,  7.8974056e-01,
        1.7716237e+00,  5.6958981e+00, -3.4061915e-01,  2.0220882e-01,
      

In [50]:
b=model1.wv['民主'] 

In [51]:
# 向量可以加減乘除
d=a+b*3
d

array([ 13.990654  , -12.111562  ,  -0.33018517,   3.3650608 ,
        -7.3823843 , -23.594728  ,  21.29218   ,   0.2441982 ,
        -5.4036226 ,  10.58201   ,  -1.1638694 ,  -6.1340294 ,
         1.5550361 ,  15.637451  ,   2.9826016 ,  -1.9969385 ,
         3.6894991 , -13.552641  , -21.821224  , -16.468216  ,
         9.352181  ,   5.5091667 ,   5.90228   , -18.17016   ,
        -7.467868  ,  18.886602  ,  11.245746  , -18.281342  ,
       -27.86699   ,   4.566697  ,   0.58131075,  15.221552  ,
       -20.056358  ,   5.862406  ,  -0.08472194,   6.18945   ,
        -1.5496873 ,  20.279291  ,  -2.1351933 , -28.993235  ,
       -24.568592  ,   6.6466084 , -13.991067  ,  21.25393   ,
        -2.3232238 ,   6.4417005 ,  11.2135    ,  -0.60856116,
         1.6567225 ,  -1.0500407 , -25.74908   ,  -4.3087583 ,
        12.700454  ,  17.46258   ,  -7.0441475 ,   5.830426  ,
        14.099746  ,  10.354751  ,  11.002479  ,  -0.10140634,
       -18.012815  ,  -4.4028997 ,  15.156728  ,  16.17

## 向量與向量間的 cos 距離

In [52]:
import numpy as np
def cos_simility(a, b):
    x = np.array(a)
    y = np.array(b)
    Lx=np.linalg.norm(x)
    Ly=np.linalg.norm(y)
    cos=round(x.dot(y)/(Lx*Ly),8) 
    return cos

In [53]:
a=model1.wv['自由'] 
b=model1.wv['民主'] 
c=cos_simility(a, b)
c

0.54990536

## 字詞的相似性

In [56]:
model1.wv.similarity('自由','民主')


0.5499054

## 最接近的字詞

In [23]:
model1.wv.most_similar('毛泽东', topn=50)

[('邓小平', 0.8402860760688782),
 ('江泽民', 0.7258259057998657),
 ('大钊', 0.7220970392227173),
 ('胡志明', 0.7058353424072266),
 ('斯大林', 0.7053905725479126),
 ('中国共产党', 0.6911027431488037),
 ('金日成', 0.6756417751312256),
 ('华国锋', 0.6740742325782776),
 ('恩来', 0.672836184501648),
 ('瞿秋白', 0.6724335551261902),
 ('陈云', 0.66998291015625),
 ('我党', 0.6644517183303833),
 ('任弼时', 0.6642876267433167),
 ('周恩来', 0.6591649651527405),
 ('闻天', 0.6567334532737732),
 ('先念', 0.6474846005439758),
 ('李大钊', 0.6465905904769897),
 ('毛主席', 0.6437059640884399),
 ('稼祥', 0.6389459371566772),
 ('片山潜', 0.6322881579399109),
 ('德怀', 0.6317355632781982),
 ('从洲', 0.6275631189346313),
 ('朱德', 0.6269260048866272),
 ('列宁', 0.6264203786849976),
 ('彭德怀', 0.6259609460830688),
 ('耀邦', 0.6248888969421387),
 ('小平同志', 0.6235034465789795),
 ('愈之', 0.6192953586578369),
 ('叶剑英', 0.6155536770820618),
 ('汉夫', 0.613665759563446),
 ('方志敏', 0.6119800209999084),
 ('伯承', 0.6118382215499878),
 ('刘少奇', 0.6067354679107666),
 ('了少奇', 0.605552375316619

In [22]:
model1.wv.most_similar('伟大领袖', topn=50)

[('伟大的领袖', 0.8727399706840515),
 ('敬爱', 0.7228566408157349),
 ('伟大的导师', 0.6712188720703125),
 ('伟大的统帅', 0.646854817867279),
 ('英明领袖', 0.6234865188598633),
 ('杜丑', 0.6193273663520813),
 ('伟大的舵手', 0.604938805103302),
 ('最红', 0.6023340225219727),
 ('伟大舵手', 0.5994139313697815),
 ('亲密战友', 0.5539891123771667),
 ('领导全党', 0.5537614822387695),
 ('陈克良', 0.5411613583564758),
 ('张寒晖', 0.5409386157989502),
 ('伟大', 0.5248295068740845),
 ('郑榕林', 0.523016095161438),
 ('光辉一生', 0.5189442038536072),
 ('舒茂贵', 0.5130596160888672),
 ('缔造者', 0.5070366263389587),
 ('韶山', 0.5038968324661255),
 ('敬仰', 0.5009653568267822),
 ('英明', 0.49674707651138306),
 ('深切关怀', 0.4937417805194855),
 ('党中央', 0.4925348162651062),
 ('朱总司令', 0.4924063980579376),
 ('永垂不朽', 0.4896009862422943),
 ('百年诞辰', 0.48583725094795227),
 ('永跟', 0.48501715064048767),
 ('无限怀念', 0.4827490746974945),
 ('周总理', 0.4817030429840088),
 ('无产阶级文艺路线', 0.48122572898864746),
 ('创立者', 0.4800833463668823),
 ('丰功伟绩', 0.4795612394809723),
 ('遗容', 0.4790158867835

In [85]:
model1.wv.most_similar('邓小平', topn=20)

[('江泽民', 0.8412005305290222),
 ('毛泽东', 0.840286135673523),
 ('小平同志', 0.7805500626564026),
 ('陈云', 0.7650571465492249),
 ('胡耀邦', 0.7251560688018799),
 ('胡锦涛', 0.7171494960784912),
 ('华国锋', 0.6806959509849548),
 ('先念', 0.6795100569725037),
 ('叶剑英', 0.6756204962730408),
 ('耀邦', 0.6753972768783569),
 ('闻天', 0.6711532473564148),
 ('陶铸', 0.6696674823760986),
 ('杨献珍', 0.662767767906189),
 ('刘少奇', 0.6580374240875244),
 ('大钊', 0.6567032337188721),
 ('张闻天', 0.6515966653823853),
 ('周恩来', 0.6490671634674072),
 ('彭真', 0.6467975378036499),
 ('云儒', 0.645505964756012),
 ('稼祥', 0.6389012932777405)]

In [92]:
model1.wv.most_similar('毛泽东', topn=20)

[('邓小平', 0.8402860760688782),
 ('江泽民', 0.7258259057998657),
 ('大钊', 0.7220970392227173),
 ('胡志明', 0.7058353424072266),
 ('斯大林', 0.7053905725479126),
 ('中国共产党', 0.6911027431488037),
 ('金日成', 0.6756417751312256),
 ('华国锋', 0.6740742325782776),
 ('恩来', 0.672836184501648),
 ('瞿秋白', 0.6724335551261902),
 ('陈云', 0.66998291015625),
 ('我党', 0.6644517183303833),
 ('任弼时', 0.6642876267433167),
 ('周恩来', 0.6591649651527405),
 ('闻天', 0.6567334532737732),
 ('先念', 0.6474846005439758),
 ('李大钊', 0.6465905904769897),
 ('毛主席', 0.6437059640884399),
 ('稼祥', 0.6389459371566772),
 ('片山潜', 0.6322881579399109)]

In [65]:
model1.wv.most_similar('革命', topn=20)

[('革命斗争', 0.7061890959739685),
 ('民主革命', 0.6715720891952515),
 ('革命战争', 0.6622600555419922),
 ('革命事业', 0.641211986541748),
 ('新民主主义革命', 0.6364023089408875),
 ('伟大', 0.6338841319084167),
 ('抗日战争', 0.6297708749771118),
 ('土地革命', 0.6213778853416443),
 ('抗日', 0.6209691166877747),
 ('国民革命', 0.6112938523292542),
 ('共产主义', 0.609893798828125),
 ('人民解放战争', 0.6095950603485107),
 ('五四运动', 0.6090352535247803),
 ('救亡图存', 0.6059989333152771),
 ('北伐战争', 0.6036877036094666),
 ('抗战', 0.5939123034477234),
 ('新文化运动', 0.5896835923194885),
 ('中国共产党', 0.5880768299102783),
 ('第一次国内革命战争', 0.5835145711898804),
 ('辛亥革命', 0.5812661647796631)]

In [66]:
model1.wv.most_similar('斗争', topn=20)

[('革命斗争', 0.7760055661201477),
 ('伟大斗争', 0.7628013491630554),
 ('长期斗争', 0.7579331994056702),
 ('英勇斗争', 0.7495409250259399),
 ('生死搏斗', 0.7249380946159363),
 ('武装斗争', 0.7183615565299988),
 ('对敌斗争', 0.6894320249557495),
 ('殊死搏斗', 0.6801496744155884),
 ('正义战争', 0.6765900254249573),
 ('战斗', 0.653632402420044),
 ('群众斗争', 0.6325584650039673),
 ('反霸斗争', 0.6245372295379639),
 ('思想斗争', 0.6206682920455933),
 ('游击战争', 0.615606427192688),
 ('殊死', 0.6152145862579346),
 ('抗争', 0.6040594577789307),
 ('人民解放战争', 0.6020842790603638),
 ('正义斗争', 0.6010591983795166),
 ('党内斗争', 0.5993779301643372),
 ('合法斗争', 0.594028115272522)]

In [67]:
model1.wv.most_similar('台湾', topn=20)

[('大陆', 0.799451470375061),
 ('台湾地区', 0.794044017791748),
 ('祖国大陆', 0.7873083353042603),
 ('台湾当局', 0.775187611579895),
 ('香港', 0.7672760486602783),
 ('台湾省', 0.7604334950447083),
 ('台北', 0.7457877993583679),
 ('中国大陆', 0.7408306002616882),
 ('岛内', 0.7268843054771423),
 ('吕秀莲', 0.714836597442627),
 ('李登辉', 0.7019785046577454),
 ('蒋帮', 0.6952126026153564),
 ('台当局', 0.6893937587738037),
 ('海峡两岸', 0.6865859031677246),
 ('台湾人', 0.6864056587219238),
 ('陈水扁', 0.6789540648460388),
 ('中国台湾', 0.6759572625160217),
 ('台澎金马', 0.6686720252037048),
 ('大陆同胞', 0.6671602129936218),
 ('两国论', 0.664807915687561)]

In [86]:
model1.wv.most_similar('共产主义', topn=20)

[('共产主义者', 0.7112361192703247),
 ('社会主义', 0.6665684580802917),
 ('科学共产主义', 0.6539430022239685),
 ('理想', 0.6444007158279419),
 ('无产阶级', 0.6279204487800598),
 ('崇高理想', 0.62421715259552),
 ('世界大同', 0.6231550574302673),
 ('共产党人', 0.6201408505439758),
 ('人生目标', 0.6161081194877625),
 ('道德', 0.6130836606025696),
 ('革命理想', 0.6111891269683838),
 ('革命', 0.6098937392234802),
 ('马克思列宁主义', 0.5976080894470215),
 ('幸福观', 0.5799585580825806),
 ('崇高', 0.5783703327178955),
 ('伟大', 0.5763978958129883),
 ('社会主义者', 0.5722209811210632),
 ('新民主主义', 0.5658215284347534),
 ('追求理想', 0.5653747916221619),
 ('革命者', 0.5653344392776489)]

## 與數個關鍵字最接近的字詞

In [89]:
a=model1.wv.most_similar(positive=['反革命','反党', '反动'],topn=20)
a

[('反社会主义', 0.7966956496238708),
 ('反人民', 0.7730640172958374),
 ('黑帮', 0.7537546157836914),
 ('右派', 0.7401278614997864),
 ('反革命两面派', 0.7167461514472961),
 ('一伙', 0.704038679599762),
 ('反社会', 0.6998015642166138),
 ('篡权', 0.6812359690666199),
 ('右派分子', 0.6806055903434753),
 ('反党集团', 0.6796138286590576),
 ('反共', 0.6778916120529175),
 ('胡风', 0.6776517629623413),
 ('反毛泽东思想', 0.6746252179145813),
 ('叛党叛国', 0.6737684011459351),
 ('反革命修正主义分子', 0.6716745495796204),
 ('罪恶', 0.6701393723487854),
 ('李世农', 0.6661509275436401),
 ('祸国殃民', 0.6660897135734558),
 ('反毛主席', 0.6584794521331787),
 ('林江', 0.6577588319778442)]

In [71]:
a=model1.wv.most_similar(['蒋党','蒋匪帮', '蒋帮','蒋贼'],topn=20)
a

[('蒋匪', 0.8415529727935791),
 ('美蒋', 0.781761109828949),
 ('蒋家', 0.7569612860679626),
 ('蒋美', 0.7271449565887451),
 ('蒋介石', 0.7267423868179321),
 ('伪政府', 0.7260276675224304),
 ('反攻大陆', 0.7226721048355103),
 ('何应钦', 0.7121812701225281),
 ('廖文毅', 0.7029813528060913),
 ('孙立人', 0.6995530724525452),
 ('陈诚', 0.6993465423583984),
 ('蒋记', 0.6911243200302124),
 ('蒋邦', 0.6898446679115295),
 ('残余匪帮', 0.6839451193809509),
 ('孙运璇', 0.6830462217330933),
 ('陈立夫', 0.6825380921363831),
 ('介石', 0.6820722222328186),
 ('台北', 0.681085467338562),
 ('蒋阎', 0.6787602305412292),
 ('共匪', 0.6724078059196472)]

## 與某篇文章最接近的字詞
* 文章內各字加總向量成為文章向量。
* 有更精確的做法 doc2vec
* 向量相加，因為最少次數，以及bigram設定，有些字詞會消失

In [81]:
import random
d=random.randint(0,len(df.token))
doc=df.token[d].split()
t=0
pass_count=0
# 向量相加，因為最少次數，以及bigram設定，有些字詞會消失
for w in doc:
    try:
        doc_wv=model1.wv[w]
        t=model1.wv[w]+t
    except: 
        # 有些字詞會消失 !!!!!
        pass_count+=1
        pass 
print (" ".join(doc))
print ('pass:', pass_count)

# 由加總的向量找最接近的字詞
model1.wv.similar_by_vector(t, topn=20)

本报 评论员 “ 我国 是 社会主义 法治 国家 ， 法律 的 尊严 和 权威 不容 践踏 。 不论 涉及 到 谁 ， 只要 触犯 法律 ， 都 将 依法处理 ， 决不 姑息 。 ” 日前 ， 中共中央 对 薄熙来 同志 严重 违纪 问题 立案 调查 公安机关 公布 对 尼尔 ・ 伍德 死亡 案 进行 依法 复查 结果 并 将 犯罪 嫌疑人 移送 司法机关 ， 得到 了 广大党员 群众 的 衷心 拥护 ， 自觉遵守 党纪国法 ， 成为 全党全国 人民 的 普遍 共识 ， 凝聚 着 改革 发展 稳定 的 强大 力量 。 无论是 作出 依法 依纪 予以 彻底 查清 的 果断 决策 ， 还是 贯彻 法律 面前 人人平等 的 法治 原则 ， 无论是 采取 正确 有力 的 各项措施 ， 还是 将 相关 情况 及时 向 社会 公布 ， 中央 的 处理 决定 ， 既 是 落实 从严治党 的 根本 要求 ， 也 是 维护 社会主义 法治 的 必然 之 举 。 同时 也 充分说明 ， 代表 人民 群众 根本利益 的 中国共产党 ， 决不允许 有 凌驾于 党纪国法 之上 的 “ 特殊 党员 ” ； 法律 面前 人人平等 ， 制度 面前 没有特权 制度 约束 没有 例外 ， 任何人 触犯 法律 都 将 受到 严肃 追究 和 严厉 惩处 。 国 无法 不 治 ， 民 无法 不立 。 一切 组织 和 个人 都 必须 严格遵守 法律 ， 这 是 现代 法治 社会 的 基本 要求 。 依法治国 ， 是 我们 党 领导 人民 治理 国家 的 基本 方略 ， 是 国家 长治久安 的 重要 保证 。 作为 我们 这样 一个 发展 中 大国 ， 大力 弘扬 法治 精神 ， 增强 全体 公民 的 法律意识 ， 对于 稳定 社会秩序 维护 国家 利益 保障 公民权利 ， 具有 至关重要 的 意义 。 当前 ， 我国 正 处于 社会转型 和 经济 结构调整 的 重要 时期 ， 面对 体制 转轨 社会转型 思想 多样 利益 多元 的 新 形势 ， 只有 所有 社会 成员 都 自觉遵守 法律 维护 法律 尊严 ， 才能 维护 正常 社会秩序 创造 良好 社会 氛围 ， 才能 更好 地 化解矛盾 纠纷 维护 社会 稳定 ， 也 才能 谈得上 实现 公平正义 推进 社会 和谐 。 严密 的 组织纪律性 ， 是 我们 

[('的', 0.737787663936615),
 ('滑逃', 0.7004714012145996),
 ('左家滩', 0.6908171772956848),
 ('，', 0.6894734501838684),
 ('铸箭', 0.6850659847259521),
 ('知相', 0.6815136671066284),
 ('难惑', 0.680069088935852),
 ('危是', 0.6798760890960693),
 ('社会', 0.6794376969337463),
 ('一拥而前', 0.6793079376220703),
 ('应不为', 0.6784835457801819),
 ('有价重', 0.6776536703109741),
 ('甜家', 0.6727809906005859),
 ('郑可俊', 0.6708317995071411),
 ('2149.5', 0.6700530052185059),
 ('外简内明', 0.6698453426361084),
 ('刘绍源', 0.6696285009384155),
 ('吟鉴', 0.6663443446159363),
 ('城始南', 0.66580730676651),
 ('和布新', 0.6650026440620422)]

## 由向量尋找最新近的詞

In [84]:
a=model1.wv['党和国家']
b=model1.wv['中共中央']
c=a+b
model1.wv.similar_by_vector(c, topn=20)

[('中共中央', 0.892673134803772),
 ('党和国家', 0.8779749870300293),
 ('党中央', 0.8031365871429443),
 ('中央', 0.750781774520874),
 ('党', 0.7446473836898804),
 ('我党', 0.7018079161643982),
 ('中共中央国务院', 0.6876002550125122),
 ('人民政协', 0.6786529421806335),
 ('中国共产党', 0.6732871532440186),
 ('中央军委', 0.6725732684135437),
 ('中央人民政府', 0.6690187454223633),
 ('党和政府', 0.6611127257347107),
 ('叶剑英', 0.6594966053962708),
 ('军委', 0.6581147909164429),
 ('我党我军', 0.65738445520401),
 ('中共中央书记处', 0.6539018154144287),
 ('邓小平', 0.6524552702903748),
 ('中央书记处', 0.6462705731391907),
 ('国务院', 0.6327842473983765),
 ('中央纪律检查委员会', 0.6211761832237244)]

## 文章與文章間的相似性

In [75]:
# Compute cosine similarity between two sets of words.
a=['邓小平','經國']
b=['英九','登輝','阿扁']
model2.n_similarity(a, b)

0.19181541178489003

In [76]:
# Compute cosine similarity between two sets of words.
a=['中正','經國']
b=['英九','登輝','阿扁']
model1.n_similarity(a, b)

0.99550812236763386

## 最不匹配的字

In [77]:
a=['建設','民主','人權','自由2','憲法','環保','社會','經濟','政治','環保']
model2.doesnt_match(a)

'憲法'

# 類比

In [99]:
model1.wv.most_similar(positive=['毛泽东','蒋介石'], negative=['中央人民政府'], topn=10)

[('汪精卫', 0.6259408593177795),
 ('鲁迅', 0.5921456217765808),
 ('大钊', 0.5863993763923645),
 ('秋白', 0.5834972262382507),
 ('瞿秋白', 0.5813263058662415),
 ('机会主义', 0.5753313899040222),
 ('阎锡山', 0.5736830234527588),
 ('孙中山', 0.5703698396682739),
 ('胡适', 0.5679365992546082),
 ('方志敏', 0.5656961798667908)]

## 刪除model, 釋放記憶體

In [ ]:
del model1


# 加入 people 模組 （待補）

In [1]:
import people


    歡迎進入 people 模組，函數列表
    資料選取
        (df, df_year)= people.doc_select(year, path, title, kw, field, unit)
        df= people.read_doc_select(path, unit, field, title)  
        df_year= people.read_doc_select_year(path, unit, field, title)  
        people.plot_doc_select_year(df_year)
        people.print_doc_select_random(df,num,title,field)
        df= people.read_doc_select_pkl(path, unit, field,title)  
    詞頻與詞對
        df= people.key_word_trend_year(kw, year,path)   
        df_corpus= people.corpus_map_year(df,title, kw, corpus,path,window)
        df_corpus= people.read_corpus_map_year(path,title)
        df_pn= people.sentiment(path, df, title)
        df_pn= people.read_sentiment(path, df, title)
    主題分析
        (df_topic, df, df_lsa)= people.topic model(doc_list,num_i, method)  
        ts= people.plot_topic_model_trend(year, df_lsa,topics, num_i) 
        (df_trend, size)= people.plot_topic_model_pei(topics, ts)
        (df_trend, size)= people.plot_topic_model_bar(t

# lsa 潛在語意分析

In [1]:
from string import *
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import numpy as np
from scipy import spatial
import pandas as pd

In [2]:
# 讀取檔案 #######################################
import pandas as pd
# 必須要有的參數
unit="arti"
field="tfidf"
path="D:/my python/people/" 
title="all"
fname=path+"working/"+unit+"_"+field+"_"+title+".pkl"
df=pd.read_pickle(fname)

##  extract_feature(path, df, num_i, token,m)
* 僅保留 tfidf 值較大者，簡化文本
* num_i 降維 
* m=0.05


In [4]:
# doc_list=df.token
def extract_feature(path, df, doc_list,m):
    # tfidf 加權
    transformer = TfidfVectorizer()
    tfidf = transformer.fit_transform(doc_list)     # sparse matrix
    # 如果文章筆數、字詞數太多，會超過記憶體 error
    dtm_tfidf=tfidf.toarray() 
    # 字詞的編號
    terms=vectorizer.vocabulary_
    # 字詞的 list
    term_list=vectorizer.get_feature_names_out()
    # 轉成 DataFrame
    df_tfidf=pd.DataFrame(dtm_tfidf, columns=term_list)

    # 只保留 TFIDF 太於 0.05 者，簡化字詞數，否則矩陣太大
    featureset = []
    for i in range(len(df_tfidf)):    
        # 所有字詞，在該篇文章中，tfidf 大於 m 為 true，否則為 false
        cri = df_tfidf.iloc[i]>=m
        # 合乎條件的字詞 list
        te = [x for j,x in enumerate(term_list) if cri[j]]    
        # 合乎條件的 tfidf list
        tf= [x for j,x in enumerate(df_tfidf.iloc[i]) if cri[j]]        
        # 合於條件的字詞改為空白隔開的string
        features_string=" ".join(te)     
        # 一篇文章一個字詞的 list
        featureset.append(features_string) 
    df['feature']=fetureset   
    return df


## 最接近的字詞

## (df_concept, transformer, lsa, term_list)= lsa_concept(path, df, token, num_i)

In [5]:
def lsa_concept(path, df, token, num_i):
    # tfidf 加權
    num_i=50
    doc_list=df[token].str.replace(","," ")

    # Term document frequency 轉換，成為 matrix ,
    transformer = TfidfVectorizer()
    tfidf = transformer.fit_transform(doc_list)     # sparse matrix
    # 字詞的編號
    terms=transformer.vocabulary_
    # 字詞的 list
    term_list=transformer.get_feature_names_out()
    
    # latent semantic analysis, 變數縮減
    svd = TruncatedSVD(num_i, algorithm = 'arpack')   # object
    lsa = svd.fit_transform(tfidf.T)  # numpy array, (number of terms * num_i)     

    dtm_lsa = svd.fit_transform(tfidf)  
    dtm_com = svd.fit(tfidf)  # object, not numpy array
    # components
    comp=dtm_com.components_
    print("維度數",len(comp))
    print("字詞數",len(comp[0]))
    
    # 每個維度軸都是各字詞的線性組合，每個維度的構成
    df_concept=pd.DataFrame(comp,columns= term_list)
    df_concept=df_concept.transpose()
    # 各維度字詞，依重要性大小排序
    con=[]
    sig=[]
    for n in range(len(df_concept.columns)):
        # 由大至小排序
        df_order= df_concept.sort_values(by=n, ascending=False)
        con.append(list(df_order.index)[0:500])
        sig.append(list(df_order[n])[0:500])
    # 印出各軸的重要字詞
    for n in range(len(con)):
        print (n,con[n][:30])          
    # 轉成 dataframe 格式
    df_concept=pd.DataFrame(con).T
    return df_concept, transformer, lsa, term_list

#  最接近的 k 個
def similarity(a,b):
    result = 1 - spatial.distance.cosine(a, b)
    return result
def kClosestTerms(k,term,transformer,lsa,term_list):     
    # 字詞的編號
    index = transformer.vocabulary_[term] 
    # 該字詞與其他字詞的相似度
    similar=[]
    for i in range(len(lsa)):
        s=similarity(lsa[index],lsa[i])
        similar.append(s) 
        if i%10000==0:
            print (i,"done",end=" ")
    closestTerms = {}   # dictionary
    for i in range(len(lsa)):
        closestTerms[term_list[i]] = similar[i]    
        if i%10000==0:
            print (i,"done",end=" ")                   
    sortedList = sorted(closestTerms , key= lambda l : closestTerms[l])
    return sortedList[::-1][0:k]

In [9]:
num_i=50
token='tfidf'
(df_concept, transformer, lsa, term_list)=concept(path, df, token, num_i)
df_concept.iloc[:20]

維度數 50
字詞數 1967779
0 ['中国', '发展', '国家', '经济', '北京', '建设', '国际', '合作', '政府', '关系', '会议', '美国', '全国', '访问', '代表团', '组织', '支持', '世界', '社会', '领导', '和平', '地区', '政治', '生产', '会见', '解决', '斗争', '政策', '提高', '总统']
1 ['生产', '提高', '建设', '群众', '干部', '企业', '领导', '部门', '管理', '农村', '全国', '思想', '学习', '组织', '农民', '教育', '服务', '农业', '技术', '改革', '社会', '制度', '职工', '增长', '产品', '经验', '市场', '质量', '生活', '研究']
2 ['斗争', '反对', '革命', '侵略', '胜利', '美帝国主义', '帝国主义', '美国', '战争', '毛主席', '团结', '谴责', '无产阶级', '和平', '阴谋', '全世界', '民族', '战斗', '苏联', '越南', '独立', '声明', '敌人', '修正主义', '军队', '解放', '力量', '民主', '群众', '毛泽东思想']
3 ['革命', '同志', '毛主席', '代表团', '学习', '思想', '无产阶级', '群众', '领导', '率领', '中国', '团长', '友好', '干部', '会见', '毛泽东思想', '北京', '乘飞机', '访问', '大使', '友谊', '胜利', '今天下午', '修正主义', '教导', '文化大革命', '教育', '到达', '友好访问', '参观']
4 ['发展', '关系', '合作', '两国', '经济', '政治', '领域', '两国关系', '友好合作', '稳定', '社会', '维护', '努力', '建设', '中方', '交流', '两国人民', '会见', '共同努力', '深化', '和平', '建交', '互利', '交往', '思想', '改革开放', '江泽民', '事业', '政策', '双边关系']
5 ['比赛', '中国', '世界', 

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,中国,生产,斗争,革命,发展,比赛,会议,美帝国主义,传真照片,传真照片,...,越南,贸易,全国,会议,朝鲜,中国,国际,联合国,出版,国家
1,发展,提高,反对,同志,关系,中国,社会,企业,两国人民,经济,...,公司,正式,美国,美国,北京,上海,朝鲜,农村,总统,朝鲜
2,国家,建设,革命,毛主席,合作,世界,制度,建设,10,国际,...,学生,经济,举办,北京,地区,谈话,友谊,农民,周年,会议
3,经济,群众,侵略,代表团,两国,北京,政府,帝国主义,附图片,企业,...,图为,部长,社会,总统,经济,计划,组织,结束,北京,出版
4,北京,干部,胜利,学习,经济,选手,代表团,民族,友谊,乘飞机,...,出席,会议,中国,全国,举办,改革,联合国,日本,经济,朝鲜民主主义人民共和国
5,建设,企业,美帝国主义,思想,政治,举办,中央,侵略,农民,无产阶级,...,苏联,协定,政府,友好,发展,结束,增长,朝鲜,解决,日本
6,国际,领导,帝国主义,无产阶级,领域,冠军,乘飞机,友谊,两国,美国,...,柬埔寨,签订,联合,发展,领导,会议,英国,建设,庆祝,平壤
7,合作,部门,美国,群众,两国关系,国际,法律,越南,总理,革命,...,通讯社,访问,经济,谈话,朝鲜民主主义人民共和国,图为,两国人民,农业,联合,阿拉伯
8,政府,管理,战争,领导,友好合作,女子,监督,斗争,地区,北京,...,学校,合作,越南,选举,平壤,服务,出版,会见,纪念,机场
9,关系,农村,毛主席,率领,稳定,运动员,北京,全世界,群众,产品,...,市场,日至,事业,建设,工业,参观,一行,总统,出版社,金日成


In [10]:
k=50
term='伟大领袖'
kClosestTerms(k,term,transformer,lsa,term_list)

0 done 10000 done 20000 done 30000 done 40000 done 50000 done 60000 done 70000 done 80000 done 90000 done 100000 done 110000 done 120000 done 130000 done 140000 done 150000 done 160000 done 170000 done 180000 done 190000 done 200000 done 210000 done 220000 done 230000 done 240000 done 250000 done 260000 done 270000 done 280000 done 290000 done 300000 done 310000 done 320000 done 330000 done 340000 done 350000 done 360000 done 370000 done 380000 done 390000 done 400000 done 410000 done 420000 done 430000 done 440000 done 450000 done 460000 done 470000 done 480000 done 490000 done 500000 done 510000 done 520000 done 530000 done 540000 done 550000 done 560000 done 570000 done 580000 done 590000 done 600000 done 610000 done 620000 done 630000 done 640000 done 650000 done 660000 done 670000 done 680000 done 690000 done 700000 done 710000 done 720000 done 730000 done 740000 done 750000 done 760000 done 770000 done 780000 done 790000 done 800000 done 810000 done 820000 done 830000 done 840000

['伟大领袖',
 '战无不胜',
 '对帝',
 '伟大红旗',
 '红卫兵',
 '光辉思想',
 '红彤彤',
 '红代会',
 '无限热爱',
 '林副主席',
 '毛主席',
 '千秋万代',
 '铁打江山',
 '战斗历程',
 '文化大革命',
 '四十八年',
 '风雷动',
 '最红',
 '万万岁',
 '无限信仰',
 '永世',
 '一万遍',
 '永不变色',
 '光焰',
 '最最',
 '红太阳',
 '与帝',
 '宝书',
 '海枯石烂',
 '无限崇拜',
 '伟大号召',
 '语录',
 '万寿无疆',
 '东风浩荡',
 '大长',
 '指路明灯',
 '一千遍',
 '无际',
 '毛泽东思想',
 '喜讯传来',
 '千遍万遍',
 '反修',
 '十六条',
 '举得',
 '九大',
 '大风大浪',
 '革命派',
 '大灭',
 '无产阶级',
 '伟大的统帅']

# word2vec

## model=people.load_word2vec(path, model)

In [18]:
def read_word2vec(path, model):
    # 執行時記錄時間、位置與訊息。
    import logging
    #logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    import gensim
    model = gensim.models.Word2Vec.load(fname)  # you can continue training with the loaded model!
    return model

In [19]:
path="D:/my python/people/" 
fname=path +"\working\w2v_model.txt"
model=read_word2vec(path, fname)

## vec=people.wv(kw)

In [4]:
def wv(kw):
    vec=model.wv[kw] 
    return vec

In [5]:
kw='自由'
vec=wv(kw)
vec

array([-7.4103341e-02, -2.8792534e+00, -2.9030464e+00,  1.9297727e+00,
       -4.1995859e+00,  1.3434430e+00,  5.3185129e+00,  1.8856300e+00,
        9.9386084e-01,  3.9040930e+00, -1.8884244e+00, -1.1290009e+00,
       -3.1349480e-01, -1.8501260e+00,  1.0676432e+00,  2.9322159e+00,
       -6.3640994e-01, -5.1550965e+00, -4.3209486e+00,  9.5664543e-01,
        2.1225898e+00,  5.4785538e+00,  3.2091327e+00, -4.4638085e+00,
       -5.6119280e+00, -5.5335873e-01,  7.6833628e-03, -1.7482197e+00,
       -3.3751745e+00,  4.2273159e+00,  1.0909537e+00,  4.0444779e+00,
       -7.0587349e+00,  2.3918550e+00, -2.6788953e-01,  6.6266918e-01,
       -2.3299170e+00,  5.6378150e+00,  5.7518399e-01, -8.3680096e+00,
       -7.8566394e+00, -1.7519004e+00, -2.4536273e+00,  5.0473399e+00,
        3.1660731e+00, -2.5422773e-01,  1.8931571e+00, -2.0494962e+00,
       -3.2127509e+00,  1.7837901e+00, -8.7715740e+00,  7.8974056e-01,
        1.7716237e+00,  5.6958981e+00, -3.4061915e-01,  2.0220882e-01,
      

## people.similar_by_vector(vec, topn)

In [6]:
def similar_by_vector(vec, topn):
    similar=model.wv.similar_by_vector(vec, topn) 
    return similar

In [7]:
similar_by_vector(vec, 20)

[('自由', 1.0),
 ('民主自由', 0.749008297920227),
 ('权利', 0.7337026596069336),
 ('自由民主', 0.7203790545463562),
 ('言论自由', 0.7055898308753967),
 ('尊重人权', 0.6780827641487122),
 ('自决', 0.6777547001838684),
 ('自由权', 0.6672273874282837),
 ('平等权利', 0.6616188883781433),
 ('自决权', 0.6532977223396301),
 ('平等', 0.6526392102241516),
 ('公民自由', 0.6461496949195862),
 ('尊严', 0.639614999294281),
 ('独立', 0.637828528881073),
 ('新闻自由', 0.625384509563446),
 ('合法', 0.6024500131607056),
 ('主权', 0.6019275784492493),
 ('民族自决权', 0.598908543586731),
 ('民族自决', 0.5933929681777954),
 ('结社', 0.592240571975708)]

## people.similar_by_doc(token, topn)

In [8]:
def similar_by_doc(token, topn):
    vec=0
    pass_count=0
    # 向量相加，因為最少次數，以及bigram設定，有些字詞會消失
    for w in token:
        try:
            token_wv=model.wv[w]
            vec=model.wv[w]+vec
        except: 
            # 有些字詞會消失 !!!!!
            pass_count+=1
            pass 
    print ('pass:', pass_count)
    similar=model.wv.similar_by_vector(vec, topn=topn)
    return similar

In [9]:
token=['自由','民主']
similar_by_doc(token, 20)

pass: 0


[('民主', 0.8880746960639954),
 ('自由', 0.8723101019859314),
 ('民主自由', 0.8000436425209045),
 ('自由民主', 0.7829821109771729),
 ('尊重人权', 0.7285253405570984),
 ('言论自由', 0.6834111213684082),
 ('权利', 0.6751291751861572),
 ('民主制度', 0.6740483641624451),
 ('民主权利', 0.6731681823730469),
 ('自决', 0.6720693707466125),
 ('平等', 0.6539479494094849),
 ('民主化', 0.645734429359436),
 ('新闻自由', 0.6349923610687256),
 ('代议制', 0.6308178305625916),
 ('公民自由', 0.6195918917655945),
 ('独立', 0.6179970502853394),
 ('公民权利', 0.6162161827087402),
 ('民族自决', 0.6143051385879517),
 ('平等权利', 0.61353999376297),
 ('民族自决权', 0.6122533082962036)]

## people.most_similar(kw)

In [10]:
def most_similar(kw):
    vec=model.wv.most_similar(kw)
    return vec

In [11]:
kw=['自由','民主']
most_similar(kw)

[('民主自由', 0.8016704320907593),
 ('自由民主', 0.7841441631317139),
 ('尊重人权', 0.7298718094825745),
 ('言论自由', 0.687035858631134),
 ('权利', 0.6799558997154236),
 ('自决', 0.6750860214233398),
 ('民主权利', 0.6720375418663025),
 ('民主制度', 0.6684405207633972),
 ('平等', 0.6566508412361145),
 ('民主化', 0.6390452980995178)]

## people.wv_similarity(kw1,kw2)

In [12]:
def wv_similarity(kw1,kw2):
    vec=model.wv.similarity(kw1,kw2)
    return vec

In [13]:
kw1='毛泽东'
kw2='蒋介石'
wv_similarity(kw1,kw2)

0.29228103

In [14]:
kw1=wv('毛泽东')
kw2=wv('蒋介石')
people.similarity(kw1,kw2)

NameError: name 'spatial' is not defined

## people.n_similarity(kw1,kw2)
* list 與 list 之間的相似性

In [ ]:
def n_similarity(kw1,kw2):
    vec=model.wv.n_similarity(kw1,kw2)
    return vec

In [ ]:
# Compute cosine similarity between two sets of words.
kw1=['自由','民主']
kw2=['革命']
model.wv.n_similarity(kw1, kw2)

# 類比

In [17]:
model.wv.most_similar(positive=['毛泽东','蒋介石'], negative=['中央人民政府'], topn=10)

[('汪精卫', 0.6259408593177795),
 ('鲁迅', 0.5921456217765808),
 ('大钊', 0.5863993763923645),
 ('秋白', 0.5834972262382507),
 ('瞿秋白', 0.5813263058662415),
 ('机会主义', 0.5753313899040222),
 ('阎锡山', 0.5736830234527588),
 ('孙中山', 0.5703698396682739),
 ('胡适', 0.5679365992546082),
 ('方志敏', 0.5656961798667908)]

## 刪除model, 釋放記憶體

In [15]:
remove model

SyntaxError: invalid syntax (Temp/ipykernel_20696/2247303887.py, line 1)